# Putting it all together using LangGraph (experimental)

**Last notebook we created a brain langchain agent that acts as a supervisor of the experts (other agents with tools).** This concept of multi-agent architectures is a current field of high research interest.

**Graphs are important in multi-agent systems** as they efficiently represent the interactions and relationships between different agents:
- **Nodes (or Vertices):** Each agent, can perform specific tasks or make decisions.
- **Edges:** Signify communication paths or decision flows between agents.

This structure enables the division of complex problems into smaller, manageable tasks, where each agent can focus on a particular aspect. The advantages of this approach include:
- Improved specialization and parallelization.
- More robust and scalable solutions.

**In our context, we can use the graph-based architecture** to build systems where agents coordinate through a defined protocol of interactions, enhancing both performance and flexibility. For instance, if one agent fails or generates suboptimal results, other agents in the system can take over or adjust their strategies accordingly, thus ensuring continuity and efficiency.

[**LangGraph**](https://python.langchain.com/docs/langgraph/), specifically designed for building multi-agent systems with LLMs, leverages these graph concepts by allowing developers to define states and transitions explicitly. It treats each agent as a state machine, with transitions defined by the possible actions an agent can take in response to its environment or the inputs it receives. LangGraph provides tools for building these systems, where each node in the graph can be an agent capable of making decisions or performing actions based on the shared state of the system. The library supports creating complex workflows where agents interact through a shared state, making it easier to develop, test, and maintain each component in isolation before integrating them into the broader system.

---


In [1]:
import os
import operator
from typing import TypedDict, Annotated, Sequence, Union, List
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.manager import CallbackManager

from langgraph.prebuilt import ToolExecutor, ToolNode
from langgraph.graph import StateGraph, MessageGraph, END


#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent,
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT, CUSTOM_CHATBOT_PREFIX

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [2]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Define our Model

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, 
#                        max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


### Define the Tools

In [4]:
doc_indexes = ["cogsrch-index-files", "cogsrch-index-csv"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           verbose=False)

In [5]:
book_indexes = ["cogsrch-index-books"]
book_search = DocSearchAgent(llm=llm, indexes=book_indexes,
                           k=10, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="booksearch",
                           description="useful when the questions includes the term: booksearch",
                           verbose=False)

In [6]:
# BingSearchAgent is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchAgent(llm=llm, k=5,
                             name="bing",
                             description="useful when the questions includes the term: bing",
                             verbose=False)

In [7]:
## CSVTabularAgent is a custom Tool class crated to Q&A over CSV files
file_url = "./data/all-states-history.csv"
csv_search = CSVTabularAgent(path=file_url, llm=llm,
                             name="csvfile",
                             description="useful when the questions includes the term: csvfile",
                             verbose=False)

In [8]:
## SQLDbAgent is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLSearchAgent(llm=llm, k=30,
                            name="sqlsearch",
                            description="useful when the questions includes the term: sqlsearch",
                            verbose=False)

In [9]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm,
                             name="chatgpt",
                            description="useful when the questions includes the term: chatgpt",
                            verbose=False)

In [10]:

tools = [www_search, sql_search, doc_search, csv_search, chatgpt_search, book_search]


### Bind the Tools to the Model

We should make sure the model knows that it has these tools available to call. We can do this by converting the LangChain tools into the format for OpenAI tool calling, and then bind them to the model class.

In [11]:
llm_with_tools = llm.bind_tools(tools) 

Let's test our model with a few messages to see if effectively calls the function sqlsearch

In [12]:
llm_with_tools.invoke([HumanMessage(content="what is your name"), 
              AIMessage(content='My name is Assistant. How can I assist you today?'),
              HumanMessage(content="sqlsearch, how many deaths in the east coast?")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CKVCMjluxcXCHpCgxCZmPdvT', 'function': {'arguments': '{"query":"SELECT SUM(deaths) FROM east_coast"}', 'name': 'sqlsearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 448, 'total_tokens': 470}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-06ba45e4-d5e1-4d40-a649-18f9e7fb4538-0', tool_calls=[{'name': 'sqlsearch', 'args': {'query': 'SELECT SUM(deaths) FROM east_coast'}, 'id': 'call_CKVCMjluxcXCHpCgxCZmPdvT'}])

This looks correct!,  the LLM responded with a AIMessage that has no content, but instead has a **tool_call**.

### Define agent state

A graph is parameterized by a state object that it passes around to each node. Each node then returns operations to update that state. These operations can either SET specific attributes on the state (e.g. overwrite the existing values) or ADD to the existing attribute. Whether to set or add is denoted by annotating the state object you construct the graph with.

For our case, the state we will track will just be a list of messages. We want each node to just add messages to that list. Therefore, we will use a TypedDict with one key (messages) and annotate it so that the messages attribute is always added to with the second parameter (operator.add).

In [13]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### Define the nodes

We now need to define a few different nodes in our graph. In langgraph, **a node can be either a function or a runnable**. There are two main nodes we need for this:

**The agent**: responsible for deciding what (if any) actions to take.
A function to invoke tools: if the agent decides to take an action, this node will then execute that action.
We will also need to define some edges. Some of these edges may be conditional. The reason they are conditional is that based on the output of a node, one of several paths may be taken. The path that is taken is not known until that node is run (the LLM decides).

**Conditional Edge**: after the agent is called, we should either:

    a. If the agent said to take an action, then the function to invoke tools should be called

    b. If the agent said that it was finished, then it should finish

**Normal Edge**: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [14]:
# Define the function that determines whether to continue or not
def should_continue(state):
    
    messages = state["messages"]
    
    last_message = messages[-1]
    # If there are no tool calls, then we finish
    if not last_message.tool_calls:
        return "end"
   # Otherwise if there is, we check if it's suppose to return direct the result of the tool or not (based on the prompt)
    else:
        arguments = last_message.tool_calls[0]["args"]
        if arguments.get("return_direct", False):
            return "final"
        else:
            return "continue"

    

# Define the function that calls the supervisor chain
async def supervisor_node(state):
    
    messages = state["messages"]
    
    PROMPT = ChatPromptTemplate.from_messages(
        [
            ("system", CUSTOM_CHATBOT_PREFIX),
            ("human", "{question}")
        ]
    )
    
    chain = (
        {
            "question": lambda x: x["question"],
        }
        | PROMPT
        | llm_with_tools
    )
    
    response = await chain.ainvoke({"question": messages})
    
    return {"messages": [response]}


# Define the function to execute tools
tool_node = ToolNode(tools)

### Define the Graph

In [15]:
# Define a new graph
workflow = StateGraph(AgentState)


# Define the two nodes we will cycle between
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("tools", tool_node)
# We add a separate node for any tool call where return_direct=True. The reason this is needed is that after this node we want to end, 
# while after other tool calls we want to go back to the LLM.
workflow.add_node("tools_final", tool_node)

# Set the entrypoint as `supervisor`
# This means that this node is the first one called
workflow.set_entry_point("supervisor")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `supervisor`.
    # This means these are the edges taken after the `supervisor` node is called.
    "supervisor",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Final call
        "final": "tools_final",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `supervisor`.
# This means that after `tools` is called, `supervisor` node is called next.
workflow.add_edge("tools", "supervisor")
# and from `tools_final` to END
workflow.add_edge("tools_final", END)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

### Define a utility print function

In [16]:
async def print_events(query):
    inputs = {"messages": [HumanMessage(content=query)]}
    event_names = ["LangGraph", "supervisor", "call_supervisor_chain", "should_continue", "tools", "tools_final"]

    async for event in app.astream_events(inputs, version="v1"):
        kind = event["event"]
        # print(event)
        if kind == "on_chain_start":
            if event["name"] in event_names:
                print("\n=======================\n")
                print(f"Starting: {event['name']}")
                print(event)

        elif kind == "on_chain_end":
            if event["name"] == "LangGraph":  
                print("\n=======================\n")
                # print(event)
                try:
                    printmd(event['data']['output']['supervisor']['messages'][0].content)
                except:
                    printmd(event['data']['output']['tools_final']['messages'][0].content)
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            # Empty content in the context of OpenAI means that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            if content:
                print(content, end="|")
        elif kind == "on_tool_start":
            print("\n=======================\n")
            print(f"Starting tool: {event['name']}")
            print(event)

### Use the graph

In [17]:
await print_events("Hello there, how are you? My name is Pablo Marin")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(




Starting: LangGraph
{'event': 'on_chain_start', 'run_id': 'a5a7383d-24c0-4e90-b6f2-c4cfa88e78c0', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='Hello there, how are you? My name is Pablo Marin')]}}}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': '275a19b5-9c9d-4dad-bb35-84887712a94f', 'tags': ['graph:step:1'], 'metadata': {}, 'data': {}}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': 'c19a0066-c454-41ea-8e81-5304680f2012', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='Hello there, how are you? My name is Pablo Marin'), AIMessage(content="I'm here and ready to assist you, Pablo. How can I help you today?", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 865, 'total_tokens': 884}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': 

I'm here and ready to assist you, Pablo. How can I help you today?

In [19]:
await print_events("bing, What's the oldest parrot alive, and how much longer is that than the average lifespan of a parrot?")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '07ae7cc1-cca6-4202-aae3-1e06556dce25', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content="bing, What's the oldest parrot alive, and how much longer is that than the average lifespan of a parrot?")]}}}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': '8cd7d8fc-83b1-4106-a34c-30e609bfaa02', 'tags': ['graph:step:1'], 'metadata': {}, 'data': {}}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': '2b80e2cb-8f65-415d-887d-d324eeb8681f', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content="bing, What's the oldest parrot alive, and how much longer is that than the average lifespan of a parrot?"), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tNG5QoLmkcdIPbxNv6skBjsA', 'function': {'arguments': '{"query":"oldest parrot alive"}', 'name': 'bing'}, 

The oldest living parrot is a subject of debate and uncertainty, but there are a few remarkable cases of long-lived parrots. Here are some examples:

1. **Poncho**: Poncho, a Green-winged macaw, is officially the world's oldest living parrot at 92 years of age. This is exceptional considering that most birds of Poncho's breed live to be around 50 or 60 years old, with some reaching 80 years of age with a healthy diet [[1]](https://www.oldest.org/animals/parrots/).

2. **Cookie**: Cookie, a Major Mitchell’s cockatoo, was recognized by the Guinness World Records as the oldest parrot ever. He lived to be at least 82 years and 88 days old when he passed away on 27 August 2016. Cookie's exact age was unknown when he arrived at Brookfield Zoo in May 1934, but his arrival was documented in a ledger dated May 1934, estimating his age at that time [[2]](https://www.guinnessworldrecords.com/world-records/442525-oldest-parrot-ever).

3. **Other Long-Lived Parrots**: There are other parrots with long lifespans, but their ages lack verifiable documentation. While Cookie holds the record for the oldest parrot on record, living to the age of 83, there are other parrots whose lifespans may be longer but lack verifiable documentation [[3]](https://savetheeaglesinternational.org/old-parrot/).

These examples showcase the exceptional longevity of certain parrots and the debate surrounding the world's oldest living parrot.

As for the average lifespan of a parrot, it varies depending on the species. For example, larger parrots such as macaws and cockatoos can live 60-80 years or more, while smaller parrots like budgies and lovebirds have an average lifespan of 15-20 years. The average lifespan of a parrot is significantly shorter than the ages of the oldest living parrots mentioned above.

If you have any further questions or need more information, feel free to ask!

### We can tell the app to return the results directly from the tool (without passing through the supervisor)

In [20]:
await print_events("docsearch, how diabetes affects covid? return the answer directly by setting return_direct=True")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '0aaca84b-52bc-423d-9ce7-f8858e2f522e', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='docsearch, how diabetes affects covid? return the answer directly by setting return_direct=True')]}}}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': '88725935-bc2c-4ec6-8f86-0dd828e3e50e', 'tags': ['graph:step:1'], 'metadata': {}, 'data': {}}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': 'aaf0fad4-3e10-4253-afc5-0f6d5b18bded', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='docsearch, how diabetes affects covid? return the answer directly by setting return_direct=True'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vpOibWg93pesEebmgWuk91mb', 'function': {'arguments': '{"query":"how diabetes affects COVID","return_direct":true}', 'name':

Diabetes can significantly affect the progression and prognosis of COVID-19. Individuals with diabetes are at a higher risk of severe pneumonia, tissue injury, excessive inflammation responses, and hypercoagulable state associated with dysregulation of glucose metabolism when infected with COVID-19 [[1]](https://api.elsevier.com/content/article/pii/S1056872720303962) [[2]](https://doi.org/10.1002/dmrr.3319). Additionally, diabetes patients often suffer from comorbidities that further worsen clinical outcomes. The impact of COVID-19 on individuals with diabetes is attributed to an imbalance in angiotensin-converting enzyme 2 (ACE2) activation pathways, leading to an inflammatory response and acute β-cell dysfunction, resulting in a hyperglycemic state [[1]](https://api.elsevier.com/content/article/pii/S1056872720303962).

Furthermore, patients with diabetes may experience a heightened inflammatory response, impaired immune response, and a hypercoagulable state, all of which contribute to the increased severity and mortality risk of COVID-19 in this population [[3]](https://doi.org/10.1007/s00508-020-01672-3) [[4]](https://doi.org/10.1038/s41430-020-0652-1). Serum levels of inflammation-related biomarkers such as IL-6, C-reactive protein, serum ferritin, and coagulation index are significantly higher in diabetic patients compared to those without diabetes, indicating their susceptibility to an inflammatory storm and rapid deterioration of COVID-19 [[2]](https://doi.org/10.1002/dmrr.3319).

The management of diabetes during the COVID-19 pandemic poses challenges, particularly in ensuring optimal glycemic control and adapting antidiabetic drugs and insulin therapy accordingly. Access to outpatient clinics may be limited during the pandemic, emphasizing the need for alternative treatment options, such as the implementation of novel telemedicine strategies [[3]](https://doi.org/10.1007/s00508-020-01672-3).

In summary, diabetes is considered a risk factor for the rapid progression and poor prognosis of COVID-19. Patients with diabetes require more intensive attention and specialized care to mitigate the impact of COVID-19 on their health outcomes [[1]](https://api.elsevier.com/content/article/pii/S1056872720303962) [[2]](https://doi.org/10.1002/dmrr.3319) [[3]](https://doi.org/10.1007/s00508-020-01672-3) [[4]](https://doi.org/10.1038/s41430-020-0652-1).

In [21]:
await print_events("sqlsearch, How many people died of covid in Texas in 2020? return the answer directly by setting return_direct=True")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': 'a913fbcc-f6a6-4c1b-b09a-935317e41380', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='sqlsearch, How many people died of covid in Texas in 2020? return the answer directly by setting return_direct=True')]}}}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': 'bdee0fc8-39ec-4233-9759-161a6f159fc8', 'tags': ['graph:step:1'], 'metadata': {}, 'data': {}}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': 'a19cafa3-48a8-4423-b756-8e1a7e529e1f', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='sqlsearch, How many people died of covid in Texas in 2020? return the answer directly by setting return_direct=True'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_40gOVRaqF6cFIpdEUGPgHCEL', 'function': {'arguments': '{"query":"How many people die

The total number of people who died of Covid in Texas in 2020 is 2,841,253.

Explanation:
I used the following SQL query to calculate the total number of deaths in Texas in 2020:
```sql
SELECT SUM(death) AS total_deaths
FROM covidtracking
WHERE state = 'TX' AND date LIKE '2020%'
```
This query sums the `death` column for Texas (`state = 'TX'`) where the date starts with '2020'. The result is 2,841,253.

In [22]:
await print_events("Thank you!! you are a great assistant. BTW, what is my name?")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '8ebe4f41-28d3-4971-bc37-5d1792e2ec44', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='Thank you!! you are a great assistant. BTW, what is my name?')]}}}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': '66a97a00-fb5a-4771-b20c-33c4b7cd8d6d', 'tags': ['graph:step:1'], 'metadata': {}, 'data': {}}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': '3fd41a1b-e28b-459b-bdcd-36ab686c011e', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='Thank you!! you are a great assistant. BTW, what is my name?'), AIMessage(content="I'm glad you're finding my assistance helpful! As for your name, I'm afraid I don't have that information. If there's anything else you'd like to know or discuss, feel free to ask!", response_metadata={'token_usage': {'completion_tokens': 44, 

I'm glad you're finding my assistance helpful! As for your name, I'm afraid I don't have that information. If there's anything else you'd like to know or discuss, feel free to ask!

# Summary

In this notebook, we have successfully implemented a multi-agent architecture utilizing LangGraph, marking a significant advancement in our capability to engineer complex, scalable systems. This framework enables the seamless integration of multiple agents, each performing distinct tasks, thereby facilitating more robust and intricate architectures.

**Note: Currently, this architecture does not include a memory component(still experimental)**, which is crucial for tasks that require historical data recall or context retention over time. However, the development of this memory component is underway and promises to enhance the functionality significantly. Once integrated, this memory capability will allow graphs to perform more complex tasks that require understanding and analyzing past interactions, leading to more intelligent and adaptive responses.

Stay tuned for updates!!